<a href="https://colab.research.google.com/github/dadaatom/Comp472_Project/blob/main/MaskDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!unzip Images.zip

In [3]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
# PREPROCESSING IMPORTS #
import os
import splitfolders

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from torchvision import datasets, transforms

In [5]:
# TRAINING IMPORTS #

import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [6]:
# EVALUATION IMPORTS #

import matplotlib.pyplot as plt
import itertools

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [7]:
================================ PREPROCESSING ================================

In [8]:
imagePath = "Images"
outputPath = "train_test_sets"

trainDir = outputPath+"/train"
testDir = outputPath+"/test"

classes = ["None", "N95", "Surgical", "Cloth"] #Folders should be labeled the same as these classes.

In [9]:
splitfolders.ratio(imagePath, output=outputPath, seed=0, ratio=(.8, 0.1,.1))

Copying files: 1381 files [00:01, 1028.58 files/s]


In [52]:
train_batch_size = 32
test_batch_size  = 200

img_height = 100
img_width = 100

In [83]:
transform = transforms.Compose([transforms.Resize((img_width,img_height)),
                                transforms.ToTensor(),
                                transforms.Normalize(.5,.5,.5)])

train_ds = datasets.ImageFolder(trainDir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=train_batch_size, shuffle=True)

In [70]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_ds = tf.keras.utils.image_dataset_from_directory(
  trainDir,
  seed=0,
  image_size=(img_height, img_width))

test_ds = tf.keras.utils.image_dataset_from_directory(
  testDir,
  seed=0,
  image_size=(img_height, img_width))

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=train_batch_size, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=test_batch_size, shuffle=False, num_workers=1)

Found 1100 files belonging to 4 classes.
Found 141 files belonging to 4 classes.


In [ ]:
train_ds = train_ds.unbatch()

In [28]:
imagesList = list(train_ds.map(lambda x, y: x))
labelsList = list(train_ds.map(lambda x, y: y))

In [12]:
# ================================ TRAINING ================================ #

In [48]:
num_epochs = 4
learning_rate = 0.001

In [85]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv_layer = nn.Sequential(

        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.fc_layer = nn.Sequential(
        nn.Dropout(p=0.1),
        nn.Linear(40000, 1000),
        nn.ReLU(inplace=True),
        nn.Linear(1000, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.1),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    # conv layers
    x = self.conv_layer(x)

    # flatten
    x = x.view(x.size(0), -1)

    # fc layer
    x = self.fc_layer(x)

    return x

In [86]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [87]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
    #for (images, labels) in zip(imagesList, labelsList):
      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss_list.append(loss.item())

      # Backprop and optimisation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Train accuracy
      total = labels.size(0)
      _, predicted = torch.max(outputs.data, 1)
      correct = (predicted == labels).sum().item()
      acc_list.append(correct / total)

      if (i + 1) % 100 == 0:
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),(correct / total) * 100))

model.eval()

KeyboardInterrupt: ignored

============================= PREDICTION TIME BABY!! =============================

In [ ]:
# UHh we gotta figure out this part but we need a y_true and y_pred
y_true = []
y_pred = []

================================== EVALUATION ==================================

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
confusion_matrix(y_true=y_true, y_pred=y_pred)
cm_plot_labels = ["no mask", "cloth", "surgical", "n95"]
cm = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=cm_plot_labels)

In [ ]:
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title="Confusion Matrix", cmap=plt.cm.Reds)

In [ ]:
precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")
accuracy = accuracy_score(y_true, y_pred)

print(f"precision: {precision: .2f}")
print(f"recall: {recall: .2f}")
print(f"f1: {f1: .2f}")
print(f"accuracy: {accuracy: .2f}")